In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('zipcombo.csv') #(9298, 258)
print(df.shape)
df.head() 

(9298, 257)


,label,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,256
0,6.0,-1.0,-1.0,-1.0,-1.000,-1.000,-1.000,-1.000,-0.631,0.862,...,0.304,0.823,1.000,0.482,-0.474,-0.991,-1.000,-1.000,-1.000,-1.0
1,5.0,-1.0,-1.0,-1.0,-0.813,-0.671,-0.809,-0.887,-0.671,-0.853,...,-0.671,-0.671,-0.033,0.761,0.762,0.126,-0.095,-0.671,-0.828,-1.0
2,4.0,-1.0,-1.0,-1.0,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,...,-1.000,-1.000,-1.000,-0.109,1.000,-0.179,-1.000,-1.000,-1.000,-1.0
3,7.0,-1.0,-1.0,-1.0,-1.000,-1.000,-0.273,0.684,0.960,0.450,...,-0.318,1.000,0.536,-0.987,-1.000,-1.000,-1.000,-1.000,-1.000,-1.0
4,3.0,-1.0,-1.0,-1.0,-1.000,-1.000,-0.928,-0.204,0.751,0.466,...,0.466,0.639,1.000,1.000,0.791,0.439,-0.199,-0.883,-1.000,-1.0


In [2]:
X = df[df.columns[1:]].values.copy()
y = df['label'].values.copy()

In [3]:
import gc 
import os
import numpy as np
import jax.numpy as jnp
from jax import random, lax
from tqdm import tqdm
from functools import partial
import jax

# Set memory limit for JAX
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '0.5'
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

# --- JIT-compiled polynomial kernel ---
@jax.jit
def polynomial_kernel_jax(X, Y, d):
    return (jnp.dot(X, Y.T) ** d)

# --- JIT-compiled Gaussian kernel ---
@jax.jit
def gaussian_kernel_jax(X, Y, c):
    # Broadcasting trick to compute pairwise distances
    # (X.shape = [Nx, D], Y.shape = [Ny, D])
    # distances.shape = [Nx, Ny]
    distances = jnp.sum((X[:, None, :] - Y[None, :, :]) ** 2, axis=-1)
    return jnp.exp(-c * distances)

@jax.jit
def shuffle_data(X, y, key):
    idx = random.permutation(key, len(X))
    return X[idx], y[idx]

def train_test_split(X, y, train_split, key):
    n_train = int(train_split * len(X))
    X_shuf, y_shuf = shuffle_data(X, y, key)
    return X_shuf[:n_train], X_shuf[n_train:], y_shuf[:n_train], y_shuf[n_train:]

In [4]:
class MultiKernelPerceptronJAX:
    def __init__(self, X_train, y_train, kernel, kernel_param, epochs):
        # Store parameters
        self.kernel = kernel
        self.kernel_param = kernel_param
        self.epochs = epochs
        
        # Convert data to JAX arrays
        self.X_train = jnp.asarray(X_train, dtype=jnp.float32)
        self.y_train = jnp.asarray(y_train, dtype=jnp.int32)

        # Class info
        self.classes = jnp.unique(self.y_train)
        self.n_classes = len(self.classes)
        self.n_samples = len(self.y_train)

        # Precompute the training kernel
        if self.kernel == 'p':
            self.K_train = polynomial_kernel_jax(self.X_train, self.X_train, self.kernel_param)
        else:
            self.K_train = gaussian_kernel_jax(self.X_train, self.X_train, self.kernel_param)

        # alpha has shape [n_classes, n_samples]
        self.alpha = jnp.zeros((self.n_classes, self.n_samples), dtype=jnp.float32)

    @partial(jax.jit, static_argnums=(0,))
    def predict(self, X):
        if self.kernel == 'p':
            K = polynomial_kernel_jax(X, self.X_train, self.kernel_param)
        else:
            K = gaussian_kernel_jax(X, self.X_train, self.kernel_param)
        
        # alpha.dot(K.T) => shape [n_classes, batch_size]
        confidence = jnp.dot(self.alpha, K.T)
        return jnp.argmax(confidence, axis=0)

    def train(self):
        """
        We replace Python loops with jax.lax.fori_loop.
        The Perceptron update is inherently sequential,
        but we can at least JIT-compile the loop to remove Python overhead.
        """
        # We define an 'update_fn' for a single sample
        def update_fn(i, carry):
            alpha, errors = carry
            # confidence shape = [n_classes,]
            confidence = jnp.dot(alpha, self.K_train[i])
            y_pred = jnp.argmax(confidence)
            y_true = self.y_train[i]

            mismatch = (y_pred != y_true)
            # Add 1 to alpha[y_true, i], subtract 1 from alpha[y_pred, i] if mismatch
            alpha = alpha.at[y_true, i].add(jnp.where(mismatch, 1.0, 0.0))
            alpha = alpha.at[y_pred, i].add(jnp.where(mismatch, -1.0, 0.0))

            errors = errors + jnp.where(mismatch, 1, 0)
            return (alpha, errors)

        # A single epoch function
        def epoch_fn(_, alpha):
            # We'll track errors within each epoch
            init_carry = (alpha, 0)
            (alpha_out, errors) = lax.fori_loop(
                0, self.n_samples, update_fn, init_carry
            )
            return alpha_out, errors

        # We do multiple epochs
        alpha = self.alpha
        total_errors = 0
        for _ in range(self.epochs):
            alpha, errors = epoch_fn(0, alpha)
            total_errors = errors  # final epoch's error

        # Update self.alpha
        self.alpha = alpha
        # Compute error rate for the final epoch
        error_rate = (total_errors / self.n_samples) * 100
        return error_rate

    @partial(jax.jit, static_argnums=(0,))
    def test(self, X_test, y_test):
        X_test = jnp.asarray(X_test, dtype=jnp.float32)
        y_test = jnp.asarray(y_test, dtype=jnp.int32)
        y_pred = self.predict(X_test)
        errors = jnp.sum(y_pred != y_test)
        error_rate = (errors / len(y_test)) * 100
        return error_rate

In [5]:
# Example usage
train_split = 0.8
d_list = range(1, 8)
results_file = 'perceptron_degree_results.npy'

# Check if we have saved results
if os.path.exists(results_file):
    saved_results = np.load(results_file, allow_pickle=True).item()
    train_error = saved_results['train_error']
    train_std = saved_results['train_std']
    test_error = saved_results['test_error']
    test_std = saved_results['test_std']
    print("Loaded saved results")
else:
    train_error, train_std, test_error, test_std = [], [], [], []
    key = random.PRNGKey(0)

    for d in d_list:
        single_run_train_errors, single_run_test_errors = [], []
        for run in tqdm(range(20)):
            # Split data
            key, subkey = random.split(key)
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_split, subkey)
            
            # Train model
            clf = MultiKernelPerceptronJAX(X_train, y_train, kernel='p', kernel_param=d, epochs=5)
            train_e = clf.train()
            test_e = clf.test(X_test, y_test)
            
            single_run_train_errors.append(train_e)
            single_run_test_errors.append(test_e)
        
        # Calculate statistics
        mean_train = jnp.mean(jnp.array(single_run_train_errors))
        std_train = jnp.std(jnp.array(single_run_train_errors))
        mean_test = jnp.mean(jnp.array(single_run_test_errors))
        std_test = jnp.std(jnp.array(single_run_test_errors))
        train_error.append(mean_train)
        train_std.append(std_train)
        test_error.append(mean_test)
        test_std.append(std_test)
        
        print(f'Degree {d}: Mean Train Error: {mean_train:.2f}, Mean Test Error: {mean_test:.2f}')
    
    # Save results
    np.save(results_file, {
        'train_error': train_error,
        'train_std': train_std,
        'test_error': test_error,
        'test_std': test_std
    })

# Display results
for d, tr_e, tr_s, te_e, te_s in zip(d_list, train_error, train_std, test_error, test_std):
    print(f"Degree {d}: Train Error = {tr_e:.2f} ± {tr_s:.2f}, Test Error = {te_e:.2f} ± {te_s:.2f}")

Loaded saved results
Degree 1: Train Error = 7.71 ± 0.25, Test Error = 9.35 ± 1.67
Degree 2: Train Error = 1.59 ± 0.14, Test Error = 4.47 ± 0.69
Degree 3: Train Error = 0.67 ± 0.10, Test Error = 3.70 ± 0.43
Degree 4: Train Error = 0.35 ± 0.08, Test Error = 3.34 ± 0.38
Degree 5: Train Error = 0.30 ± 0.08, Test Error = 3.33 ± 0.41
Degree 6: Train Error = 0.22 ± 0.07, Test Error = 3.16 ± 0.40
Degree 7: Train Error = 0.18 ± 0.06, Test Error = 3.73 ± 1.53


In [6]:
from tabulate import tabulate

#tabulate the results
data = []
for i in range(len(d_list)):
    train_result = f"{'{:.4f}'.format(train_error[i])}±{'{:.4f}'.format(train_std[i])}"
    test_result = f"{'{:.4f}'.format(test_error[i])}±{'{:.4f}'.format(test_std[i])}"
    result = [int(d_list[i]), train_result, test_result]
    data.append(result)
print(tabulate(data, 
               headers = ["d", "Mean Train Error Rates (%)", "Mean Test Error Rates (%)"], 
               tablefmt = "simple_outline",
               stralign = "center"))

┌─────┬──────────────────────────────┬─────────────────────────────┐
│   d │  Mean Train Error Rates (%)  │  Mean Test Error Rates (%)  │
├─────┼──────────────────────────────┼─────────────────────────────┤
│   1 │        7.7097±0.2457         │        9.3548±1.6674        │
│   2 │        1.5878±0.1406         │        4.4677±0.6874        │
│   3 │        0.6722±0.0988         │        3.7016±0.4322        │
│   4 │        0.3482±0.0757         │        3.3360±0.3834        │
│   5 │        0.3038±0.0839         │        3.3280±0.4069        │
│   6 │        0.2245±0.0690         │        3.1586±0.4048        │
│   7 │        0.1795±0.0617         │        3.7285±1.5295        │
└─────┴──────────────────────────────┴─────────────────────────────┘


In [7]:
def cross_validate_single_run(X, y, train_split=0.8, k_folds=5, d_list=range(1, 8)):
    # Get initial train/test split
    key = random.PRNGKey(np.random.randint(0, 1000))
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_split, key)
    
    # Convert to numpy for cross-validation
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    best_cv_error = float('inf')
    best_d = 1
    
    n_samples = len(X_train)
    fold_size = n_samples // k_folds
    
    for d in d_list:
        cv_error = 0
        for fold in range(k_folds):
            # Clear memory before each fold
            gc.collect()
            jax.clear_caches()
            
            # Create fold indices
            val_start = fold * fold_size
            val_end = val_start + fold_size if fold < k_folds - 1 else n_samples
            
            # Split data
            val_idx = slice(val_start, val_end)
            train_idx = np.concatenate([np.arange(val_start), np.arange(val_end, n_samples)])
            
            # Train model
            clf = MultiKernelPerceptronJAX(
                X_train[train_idx], y_train[train_idx],
                kernel='p', kernel_param=d, epochs=5
            )
            clf.train()
            
            # Get validation error
            val_error = clf.test(X_train[val_idx], y_train[val_idx])
            cv_error += val_error
            
            # Clean up explicitly
            del clf
            gc.collect()
            jax.clear_caches()
            
        cv_error /= k_folds
        if cv_error < best_cv_error:
            best_cv_error = cv_error
            best_d = d
        
        # Clear memory after testing each d
        gc.collect()
        jax.clear_caches()
    
    # Final training with best d
    gc.collect()
    jax.clear_caches()
    
    final_clf = MultiKernelPerceptronJAX(
        X_train, y_train,
        kernel='p', kernel_param=best_d, epochs=5
    )
    train_error = final_clf.train()
    test_error = final_clf.test(X_test, y_test)
    
    # Clean up before returning
    del final_clf, X_train, X_test, y_train, y_test
    gc.collect()
    jax.clear_caches()
    
    return best_d, float(train_error), float(test_error)

# Run experiment with checkpointing
n_runs = 20
results_file = 'cross_validation_results.npy'

# Load previous results if they exist
if os.path.exists(results_file):
    results = np.load(results_file, allow_pickle=True).item()
    start_run = len(results['best_d'])
    print(f"\nCompleted runs:")
    for i in range(start_run):
        print(f"Run {i}: d*={results['best_d'][i]}, train={results['train_error'][i]:.2f}%, test={results['test_error'][i]:.2f}%")
    print(f"\nResuming from run {start_run}")
else:
    results = {
        'best_d': [],
        'train_error': [],
        'test_error': []
    }
    start_run = 0

# Process remaining runs
for run in tqdm(range(start_run, n_runs), desc='Processing runs'):
    try:
        # Clear memory before each run
        gc.collect()
        jax.clear_caches()
        
        best_d, train_error, test_error = cross_validate_single_run(X, y)
        results['best_d'].append(best_d)
        results['train_error'].append(train_error)
        results['test_error'].append(test_error)
        
        # Save progress after each run
        np.save(results_file, results)
        
        # Print progress and clear memory
        print(f"Run {run}: d*={best_d}, train={train_error:.2f}%, test={test_error:.2f}%")
        gc.collect()
        jax.clear_caches()
        
    except Exception as e:
        print(f"Error in run {run}: {str(e)}")
        break


Completed runs:
Run 0: d*=6, train=0.24%, test=3.66%
Run 1: d*=7, train=0.23%, test=3.23%
Run 2: d*=7, train=0.24%, test=3.06%
Run 3: d*=6, train=0.20%, test=3.17%
Run 4: d*=4, train=0.47%, test=3.39%
Run 5: d*=6, train=0.09%, test=3.06%
Run 6: d*=6, train=0.32%, test=3.55%
Run 7: d*=5, train=0.42%, test=3.66%
Run 8: d*=6, train=0.11%, test=3.12%
Run 9: d*=4, train=0.36%, test=3.44%
Run 10: d*=6, train=0.30%, test=3.55%
Run 11: d*=5, train=0.20%, test=3.60%
Run 12: d*=5, train=0.19%, test=3.98%
Run 13: d*=7, train=0.17%, test=3.23%
Run 14: d*=5, train=0.27%, test=3.55%
Run 15: d*=5, train=0.17%, test=2.80%

Resuming from run 16


Processing runs:  25%|██▌       | 1/4 [00:21<01:03, 21.02s/it]

Run 16: d*=5, train=0.15%, test=2.58%


Processing runs:  50%|█████     | 2/4 [00:40<00:40, 20.31s/it]

Run 17: d*=6, train=0.12%, test=3.87%


Processing runs:  75%|███████▌  | 3/4 [01:01<00:20, 20.48s/it]

Run 18: d*=5, train=0.16%, test=3.44%


Processing runs: 100%|██████████| 4/4 [01:22<00:00, 20.74s/it]

Run 19: d*=5, train=0.30%, test=3.12%


In [8]:
# Tabulate all results
if os.path.exists('cross_validation_results.npy'):
    results = np.load('cross_validation_results.npy', allow_pickle=True).item()
    data = []
    for i in range(len(results['best_d'])):
        test_result = f"{'{:.4f}'.format(results['test_error'][i])}"
        result = [int(i+1), results['best_d'][i], test_result]
        data.append(result)
    print(tabulate(data, 
                  headers=["Run", "d*", "Test Error Rates (%)"],
                  tablefmt="simple_outline",
                  numalign="center"))
else:
    print("No results file found")

┌───────┬──────┬────────────────────────┐
│  Run  │  d*  │  Test Error Rates (%)  │
├───────┼──────┼────────────────────────┤
│   1   │  6   │         3.6559         │
│   2   │  7   │         3.2258         │
│   3   │  7   │         3.0645         │
│   4   │  6   │         3.172          │
│   5   │  4   │         3.3871         │
│   6   │  6   │         3.0645         │
│   7   │  6   │         3.5484         │
│   8   │  5   │         3.6559         │
│   9   │  6   │         3.1183         │
│  10   │  4   │         3.4409         │
│  11   │  6   │         3.5484         │
│  12   │  5   │         3.6022         │
│  13   │  5   │         3.9785         │
│  14   │  7   │         3.2258         │
│  15   │  5   │         3.5484         │
│  16   │  5   │         2.7957         │
│  17   │  5   │         2.5806         │
│  18   │  6   │         3.871          │
│  19   │  5   │         3.4409         │
│  20   │  5   │         3.1183         │
└───────┴──────┴──────────────────